In [1]:
import sqlalchemy as db

In [2]:
engine = db.create_engine('sqlite:///kabuto_live.sqlite')
connection = engine.connect()
metadata = db.MetaData()
orders = db.Table('orders', metadata, autoload=True, autoload_with=engine)

In [3]:
query = db.select([orders])

In [4]:
ResultProxy = connection.execute(query)

In [5]:
ResultSet = ResultProxy.fetchall()

In [6]:
ResultProxy.keys()

RMKeyView(['id', 'ft_trade_id', 'ft_order_side', 'ft_pair', 'ft_is_open', 'order_id', 'status', 'symbol', 'order_type', 'side', 'price', 'average', 'amount', 'filled', 'remaining', 'cost', 'order_date', 'order_filled_date', 'order_update_date', 'ft_fee_base'])

In [13]:
{key: val for key, val in zip(ResultProxy.keys(), ResultSet[9])}

{'id': 10,
 'ft_trade_id': 2,
 'ft_order_side': 'sell',
 'ft_pair': '7140@1/JPY',
 'ft_is_open': True,
 'order_id': '20220506A01N93169139',
 'status': None,
 'symbol': None,
 'order_type': None,
 'side': None,
 'price': None,
 'average': None,
 'amount': None,
 'filled': None,
 'remaining': None,
 'cost': None,
 'order_date': datetime.datetime(2022, 5, 6, 4, 29, 3, 550612),
 'order_filled_date': None,
 'order_update_date': datetime.datetime(2022, 5, 6, 4, 29, 3, 546914),
 'ft_fee_base': None}

In [8]:
from freqtrade.persistence import Order
from sqlalchemy import text

result = connection.execute(text("""SELECT * FROM 'orders' LIMIT 0,30"""))
result

In [10]:
from freqtrade.persistence import Trade, init_db

In [49]:
from sqlalchemy.orm import scoped_session, sessionmaker, declarative_base
from sqlalchemy import create_engine
from freqtrade.enums import TradingMode
import arrow
import os

spot, margin, futures = TradingMode.SPOT, TradingMode.MARGIN, TradingMode.FUTURES

trade = Trade(
    id=1,
    pair='ADA/USDT',
    stake_amount=0.001,
    open_rate=0.01,
    amount=5,
    is_open=True,
    open_date=arrow.utcnow().datetime,
    fee_open=0.001,
    fee_close=0.001,
    exchange='binance',
    is_short=False,
    leverage=2.0,
    trading_mode=margin
)

order = Order(
    order_id=0,
    ft_order_side='buy',
    ft_pair=trade.pair,
    ft_is_open=False,
    status="closed",
    symbol=trade.pair,
    order_type="market",
    side="buy",
    price=100,
    average=100,
    filled=0.1,
    remaining=0,
    cost=0.01,
    order_date=trade.open_date,
    order_filled_date=trade.open_date,
)

db_url = f'sqlite:///debug.db'
init_db(db_url)
# https://docs.sqlalchemy.org/en/13/orm/contextual.html#thread-local-scope
# Scoped sessions proxy requests to the appropriate thread-local session.
# We should use the scoped_session object - not a seperately initialized version
Trade._session = scoped_session(sessionmaker(bind=engine, autoflush=True))
Order._session = scoped_session(sessionmaker(bind=engine, autoflush=True))
Trade.query = Trade._session.query_property()
Order.query = Order._session.query_property()

Trade.query.session.add(trade)
Order.query.session.add(order)

Trade.commit()
Order.query.session.commit()
